In [42]:
import torch
import numpy as np
from collections import defaultdict

import pandas as pd
import pathlib
from rich import print

import matplotlib.pyplot as plt

# set pyplot theme to ggplot2
plt.style.use("ggplot")
import wandb

In [43]:
source_csv_filepath = (
    "19042024_df.csv"
)
source_csv_filepath = pathlib.Path(source_csv_filepath)

In [44]:
def load_data_as_df(filepath):
    df = pd.read_csv(filepath)

    # replace NA with 5
    df = df.fillna(5)
    # rename unnamed column to model_name
    df = df.rename(columns={"Unnamed: 0": "model_name"})
    # if a columne has testing/ensemble_3/ in it then rename it to replace testing/ensemble_3/ with ''
    df.columns = df.columns.str.replace("testing/ensemble_3/", "")
    # remove column exp_name
    # df = df.drop(columns=["exp_name"])
    return df

In [45]:
df = load_data_as_df(source_csv_filepath)


# visualize using tabulate for ipython noteboooks
from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

In [46]:
def move_column(df, column_name, position):
    """
    Move a specific column to a specified position in a DataFrame.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.
    column_name (str): The name of the column to move.
    position (int): The position to move the column to.

    Returns:
    pandas.DataFrame: The modified DataFrame.
    """
    # Ensure the position is within the valid range
    if position < 0 or position > len(df.columns):
        raise ValueError(f"Position must be between 0 and {len(df.columns)}")

    # Get a list of the column names
    cols = df.columns.tolist()

    # Check if the column name is in the list
    if column_name in cols:
        # Remove the column from its current position
        cols.remove(column_name)

        # Insert the column at the specified position
        cols.insert(position, column_name)

        # Reorder the DataFrame
        df = df[cols]

    return df

df = move_column(df, 'imagenet1k.accuracy_top_1-epoch-mean', 1)  # Move 'hellobro' column to the second position
df = move_column(df, 'imagenet1k.accuracy_top_5-epoch-mean', 2)
df = move_column(df, 'imagenet1k.loss-epoch-mean', 3)

from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )


In [47]:
def rank_columns(df, keywords):
    """
    Rank the columns of a DataFrame based on keywords.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.
    keywords (dict): A dictionary mapping keywords to a boolean indicating whether to rank in
                     ascending order. For example, {'loss': True, 'acc': False}.

    Returns:
    pandas.DataFrame: The DataFrame with ranked columns.
    """
    for column in df.columns:
        for keyword, ascending in keywords.items():
            if keyword.lower() in column.lower():
               
                df[column] = df[column].rank(method='min', ascending=ascending)
                break  # Break the inner loop if keyword is found

    return df

# Define the keywords to look for in column names, and whether to rank in ascending order
keywords = {
    'loss': True,
    "mae": True,
    'acc': False,
    "miou": False,
    "auc": False,
    "bs": True,
    "aps": False,
}

df = rank_columns(df, keywords)
df = move_column(df, 'imagenet1k.accuracy_top_1-epoch-mean', 1)


In [48]:
from IPython.display import display

# Assuming df is your DataFrame
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

In [49]:
from scipy.stats import spearmanr

def reorder_columns_by_similarity(df):
    """
    Reorder the columns of a DataFrame based on their similarity to the second column,
    while keeping the first column in place.

    Parameters:
    df (pandas.DataFrame): The DataFrame to modify.

    Returns:
    pandas.DataFrame: The DataFrame with reordered columns.
    """
    # Get the first and second column names
    first_column = df.columns[0]
    second_column = df.columns[3]
    
    # normalize the columns 1 onwards
    
    

    # Calculate the Spearman rank correlation between the second column and the others
    correlations = {column: np.abs(spearmanr(df[second_column], df[column]).correlation) for column in df.columns if column != first_column}

    # Sort the columns by their correlation to the second column in descending order
    sorted_columns = sorted(correlations, key=correlations.get, reverse=True)

    # Keep the first column in its place and reorder the rest
    df = df[[first_column] + sorted_columns]

    return df

df = reorder_columns_by_similarity(df)


In [50]:
# sort rows by the average rank of the columns except the first one
df["mean_rank"] = df.iloc[:, 1:].mean(axis=1)
df = df.sort_values("mean_rank")
df = df.drop(columns=["mean_rank"])

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        df.style.set_table_attributes('style="display:inline"').bar(
            color="lightblue"
        )
    )

/var/folders/gh/_5t3nhn94t12m7rh483b8b7r0000gn/T/ipykernel_41407/2601485272.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



In [51]:
from typing import Optional
import plotly.graph_objects as go
import pandas as pd
import yaml

    
def plot_heatmap(df: pd.DataFrame, task_type: Optional[str] = None, metrics_to_include: Optional[list] = None, width=1500, height=500, name="heatmap"):
    """
    Plot a heatmap of the DataFrame using Plotly, with rows sorted by their average rank.
    The x-axis labels reflect the order of the columns after they have been reordered by similarity.

    Parameters:
    df (pandas.DataFrame): The DataFrame to plot.

    Returns:
    None
    """
    # Define the task mapping
    task_mapping = yaml.safe_load(open("task_to_metric_mapping_opt.yaml", "r"))
    if task_type:
        column_names = task_mapping[task_type] + ["imagenet1k.top_accuracy_1"]
        keep_columns = [column for column in df.columns if column in column_names]
    else:
        keep_columns = df.columns[1:]
        
    # Exclude the label column from the data to be plotted
    labels = df[df.columns[0]]
    
    data = df[keep_columns]

    # Add a temporary column to the DataFrame with the average rank of each row
    data['average_rank'] = data.mean(axis=1)
    
    # Sort the DataFrame by this temporary column
    data.sort_values('average_rank', ascending=False, inplace=True)
    print(data['average_rank'])

    # Remove the temporary column
    data.drop('average_rank', axis=1, inplace=True)

    # Update the labels to match the new order
    labels = labels[data.index]
    labels = [label.replace("-224", "") for label in labels]
    data.columns = [column.replace("image", "img").replace("text", "txt").replace("_to_", "-").replace("accuracy_top_", "acc@").replace("accuracy", "acc").replace("-epoch-mean", "").replace("flickr30k", "f30k").replace("winoground", "wg").replace("pokemonblipcaptions", "pbc").replace("newyorkercaptioncontest", "nycc").replace("image-to-text", "img-to-txt").replace("text-to-image", "txt-to-img") for column in data.columns]
    columns_by_task = [metric for datasets in task_mapping.values() for metric in datasets]
    columns_by_task = [column.replace("image", "img").replace("text", "txt").replace("_to_", "-").replace("accuracy_top_", "acc@").replace("accuracy", "acc").replace("-epoch-mean", "").replace("flickr30k", "f30k").replace("winoground", "wg").replace("pokemonblipcaptions", "pbc").replace("newyorkercaptioncontest", "nycc").replace("image-to-text", "img-to-txt").replace("text-to-image", "txt-to-img") for column in columns_by_task]
    colour_set_pastel = ["#FF6666", "#FFCC66", "#99CC99", "#66CCCC", "#6699CC", "#CC99CC", "#FF99CC", "#FFCCFF", "#CCFFFF", "#99FFCC", "#99CCFF", "#CC99FF", "#FF99FF", "#FFCC99", "#FF9999", "#FF6666", "#FF3333", "#FF0000", "#CC0000", "#990000", "#660000", "#330000", "#000000", "#333333", "#666666", "#999999", "#CCCCCC", "#FFFFFF"]
    tasks_colors = {
    "Image Classification": "#000000",  # Black
    "Few Shot Image Classification": "#003366",  # Dark Blue
    "Image Relational Reasoning": "#DC143C",  # Crimson Red
    "Image Segmentation": "#228B22",  # Forest Green
    "Image to Text Zero Shot Learning": "#F0A570",  # Charcoal Gray
    "Medical Classification": "#7851A9",  # Royal Purple
    "Medical Segmentation": "#008080",  # Teal
    "Video Classification": "#00CED1",  # Dark Turquoise
    "Video Regression": "#000080"  # Navy Blue
    }

    column_to_colour = {column.replace("image", "img").replace("text", "txt").replace("_to_", "-").replace("accuracy_top_", "acc@").replace("accuracy", "acc").replace("-epoch-mean", "").replace("flickr30k", "f30k").replace("winoground", "wg").replace("pokemonblipcaptions", "pbc").replace("newyorkercaptioncontest", "nycc").replace("image-to-text", "img-to-txt").replace("text-to-image", "txt-to-img"): tasks_colors[task] for task, columns in task_mapping.items() for column in columns}
    
    # remove columns with terms "bs-1" and corresponding values
    # remove all columns matching any of the patterns in 'terms'
    terms = ["-bs", "-aps", "-auc", "focal", "nycc.loss", "ham10k.loss", "iwildcam.loss", "wg.loss", "clevr.loss",
    'clevr.loss_colour',
    'clevr.loss_count',
    'clevr.loss_material',
    'clevr.loss_shape',
    'clevr.loss_size',
    'clevr.loss_yes_no', 'happy.acc@5', "pascal.loss", "cityscapes.dice_loss", "nyu.ce_loss", "nyu.dice_loss"]
    columns_by_task = [column for column in columns_by_task if not any(term in column for term in terms)]
    
    
    # order columns clustered by task
    data_columns = data.columns
    cityscapes_columns = [column for column in data_columns if "cityscapes" in column]
    # print(cityscapes_columns, columns_by_task)
    data = data[columns_by_task]
    
    # add blue colour to all entries in columns by task
    
    
    if metrics_to_include:
        new_columns_by_task = []
        for metric in columns_by_task:
            # print([include_metric in metric for include_metric in metrics_to_include], metrics_to_include)
            if any([include_metric in metric for include_metric in metrics_to_include]):
                new_columns_by_task.append(metric)
        columns_by_task = new_columns_by_task
        data = data[columns_by_task]
        # print(columns_by_task)
    
    columns_by_task = [f'<span style="color:{column_to_colour[column]}">{column}</span>' for column in columns_by_task]
    labels_to_modality = {"whisper": "audio", "mpnet": "audio", "bert": "text", "bart": "text", "clip-b16": "image_text", "siglip-p16": "image_text"}
    # if label not in labels_to_modality then set it to image
    labels_to_modality = {label: "image" if label not in labels_to_modality else labels_to_modality[label] for label in labels}
    modality_to_coloyr = {"audio": "#228B22", "text": "#DC143C", "image": "#000000", "image_text": "#000080"}
    labels = [f'<span style="color:{modality_to_coloyr[labels_to_modality[label]]}">{label}</span>' for label in labels]
    
    fig = go.Figure(data=go.Heatmap(
        z=data,
        x=columns_by_task,
        y=labels,
        colorscale='inferno_r',  # Use reversed Red color scale
    ))

    fig.update_layout(
        title='Model vs Dataset Rankings',
        xaxis_nticks=len(data.columns),
        yaxis_nticks=len(labels),
        
        autosize=True,
        width=width,
        height=height,
        margin=dict(
            l=50,
            r=50,
            b=100,
            t=100,
            pad=4
        )
    )
    # put legend on the top
    fig.update_layout(
        xaxis=dict(tickfont=dict(size=14), tickangle=90), # Adjust this size for your x-axis font size
        yaxis=dict(tickfont=dict(size=14)) # Adjust this size for your x-axis font size
    )

    fig.show()
    fig.write_image(f"paper/{name}.png", scale=10)
df = move_column(df, 'imagenet1k.accuracy_top_1-epoch-mean', 1)  # Move 'hellobro' column to the second position
df = move_column(df, 'imagenet1k.accuracy_top_5-epoch-mean', 2)
df = move_column(df, 'imagenet1k.loss-epoch-mean', 3)
# apply logmax softmax to the columns in a columnwise manner


plot_heatmap(df)


/var/folders/gh/_5t3nhn94t12m7rh483b8b7r0000gn/T/ipykernel_41407/2105982452.py:32: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



14    14.609137
10    13.868020
11    13.243655
15    12.761421
13    11.680203
4      9.558376
8      8.736041
6      8.664975
1      7.228426
3      6.609137
2      5.746193
5      5.395939
9      5.243655
12     4.517766
7      4.340102
0      3.593909
Name: average_rank, dtype: float64

In [53]:
plot_heatmap(df, metrics_to_include=['clevr', 'fungi', 'kinetics', 'chexpert', 'places365', 'nycc', 'diabetic', 'ucf', 'happy'], height=500, width=700, name="dev-heatmap")

/var/folders/gh/_5t3nhn94t12m7rh483b8b7r0000gn/T/ipykernel_41407/2105982452.py:32: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



14    14.609137
10    13.868020
11    13.243655
15    12.761421
13    11.680203
4      9.558376
8      8.736041
6      8.664975
1      7.228426
3      6.609137
2      5.746193
5      5.395939
9      5.243655
12     4.517766
7      4.340102
0      3.593909
Name: average_rank, dtype: float64